### Brief Description

This notebook was used for downloading temperature and precipitation data from NASA Power API. I am downloading wind, temperature and precipitation data

#### Imports

In [14]:
import requests
import pandas as pd
from pathlib import Path
from datetime import date

#### Configs

In [ ]:
## Config

LOCATIONS = {
    "Story_County_IA": {"lat_min": 41.85, "lat_max": 44.20, "lon_min": -93.81, "lon_max": -90.19},
    "McLean_County_IL": {"lat_min": 40.21, "lat_max": 42.85, "lon_min": -89.23, "lon_max": -86.47}
}

PARAMS=['PRECTOTCORR', 'T2M', 'WS10M']

START_YEAR = 2019
END_YEAR = 2025

OUTPUT_DIR = Path("C:/Users/Arnold/OneDrive/Desktop/CAPSTONE PROJECT/farming_risk_regions/data/raw")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

BASE_URL = 'https://power.larc.nasa.gov/api/temporal/daily/regional'


#### Downloading the Data

In [16]:
## Looping through parameters and locations to download data

for param in PARAMS:

    for location, coords in LOCATIONS.items():

        all_records = []

        output_file = OUTPUT_DIR / f"{location}_{param}_data.csv"

        # Looping over years

        for year in range(START_YEAR, END_YEAR):

            year_start = date(year, 1, 1)
            year_end = date(year, 12, 31) if year < END_YEAR else date(year, 8, 31) 

            try:
                # Construct API request. Refer to https://power.larc.nasa.gov/docs/v1/api/
                params = {
                    "parameters": param,
                    "community": "AG", # Agricultural community
                    "longitude-min": coords['lon_min'], 
                    "longitude-max": coords['lon_max'],
                    "latitude-min": coords['lat_min'],
                    "latitude-max": coords['lat_max'],
                    "start": year_start.strftime("%Y%m%d"),
                    "end": year_end.strftime("%Y%m%d"),
                    "format": "JSON"
                }

                response = requests.get(BASE_URL, params=params, timeout=10)
                print(f"Request URL: {response.url}")
                
                response.raise_for_status()
                data = response.json()
                
                # Parsing and storing the data

                if 'features' in data and data['features']:
                    param_data = data["features"][0]["properties"]["parameter"].get(param, {})
                    year_records = [
                        {"date": f"{timestamp[:4]}-{timestamp[4:6]}-{timestamp[6:8]}",
                         param: value if value != -999 else None}
                        for timestamp, value in param_data.items()
                    ]
                    all_records.extend(year_records)
                else:
                    print(f"Error: Missing expected data fields in response for {location}, param {param}, year {year}.")
                    print(f"Response: {str(data)[:200]}")
            
            except requests.exceptions.HTTPError as e:
                print(f"Error fetching data for {location}, param {param}, year {year}: {e}")
                continue
            
            except requests.exceptions.RequestException as e:
                print(f"Unexpected error for {location}, param {param}, year {year}: {e}")
                continue
            
            except (KeyError, IndexError) as e:
                print(f"Error: Missing expected data fields in response for {location}, param {param}, year {year}.")
                print(f"Response: {str(data)[:200]}")
                continue
        
        print(f"{location} {param}: total records for all years = {len(all_records)}")
        print(f"Intended output file: {output_file}")
        
        # Save to CSV
        if all_records:
            df = pd.DataFrame(all_records)
            df.to_csv(output_file, index=False)
            print(f"Saved {len(df)} records for {location} ({param}) to {output_file}")
            print(df.head())
        else:
            print(f"No data to save for {location} ({param}).")

Request URL: https://power.larc.nasa.gov/api/temporal/daily/regional?parameters=PRECTOTCORR&community=AG&longitude-min=-93.81&longitude-max=-90.19&latitude-min=41.85&latitude-max=44.2&start=20190101&end=20191231&format=JSON
Request URL: https://power.larc.nasa.gov/api/temporal/daily/regional?parameters=PRECTOTCORR&community=AG&longitude-min=-93.81&longitude-max=-90.19&latitude-min=41.85&latitude-max=44.2&start=20200101&end=20201231&format=JSON
Request URL: https://power.larc.nasa.gov/api/temporal/daily/regional?parameters=PRECTOTCORR&community=AG&longitude-min=-93.81&longitude-max=-90.19&latitude-min=41.85&latitude-max=44.2&start=20210101&end=20211231&format=JSON
Request URL: https://power.larc.nasa.gov/api/temporal/daily/regional?parameters=PRECTOTCORR&community=AG&longitude-min=-93.81&longitude-max=-90.19&latitude-min=41.85&latitude-max=44.2&start=20220101&end=20221231&format=JSON
Request URL: https://power.larc.nasa.gov/api/temporal/daily/regional?parameters=PRECTOTCORR&community=AG